In [ ]:
import torch
import os

from torch import nn
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.nn import Module
from torch.utils.data import Subset
import numpy as np

from tqdm import tqdm

import random

from typing import List, Dict
from sklearn.model_selection import train_test_split

import pandas as pd


from speechbrain.lobes.models.Xvector import Xvector, Classifier
from speechbrain.lobes.models import ECAPA_TDNN
from speechbrain.utils.seed import seed_everything
 
from transformers import get_linear_schedule_with_warmup

from speechbrain.nnet.pooling import AttentionPooling, StatisticsPooling

from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, normalize

import librosa

from scipy.io import wavfile





/home/stinasb/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []


In [5]:
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "caching_allocator"

In [6]:
class CustomAttentionPooling(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        
        self.input_dim = input_dim
        
        self.attn_pooling_w = torch.nn.Linear(input_dim, 1)
        
    def forward(self, x, mask=None):
        attn_weights = self.attn_pooling_w(x).squeeze(-1).float()
        
        if mask is not None:
            attn_weights[~mask] = float('-inf')
        attn_weights = torch.nn.functional.softmax(attn_weights, dim=-1).unsqueeze(-1)
        out = torch.sum(x*attn_weights, dim=1)
        return out
            

class CustomStatisticsPooling(nn.Module):
    def __init__(self, return_mean=True, return_std=True):
        super().__init__()
        self.eps = 1e-5
        self.return_mean = return_mean
        self.return_std = return_std
        
        if not (self.return_mean or self.return_std):
            raise ValueError("Enable either mean and/or std for statistical pooling.")
    
    def forward(self, x, mask=None):
        if mask is not None:
            
            
            #mask = mask.unsqueeze(-1)  # Expand to match x's shape (batch_size, time_steps, features)
            mask = mask.unsqueeze(-1).expand(-1, -1, x.size(-1))  
            masked_x = x * mask.float()
            valid_counts = mask.sum(dim=1).clamp(min=1) 
            
            if self.return_mean:
                mean = masked_x.sum(dim=1)/valid_counts

            if self.return_std:
                mean_expanded = mean.unsqueeze(1).expand(-1, x.size(1), -1)
                
                variance = ((masked_x-mean_expanded)**2*mask).sum(dim=1)/valid_counts
                std = torch.sqrt(variance+self.eps)
        else:
            if self.return_mean:
                mean = x.mean(dim=1)
                
            if self.return_std:
                std = x.std(dim=1)
                
        if self.return_mean and self.return_std:
            pooled_stats = torch.cat((mean, std), dim=1)
        elif self.return_mean:
            pooled_stats = mean
        elif self.return_std:
            pooled_stats = std
            
        return pooled_stats
    
    
    
class CustomXvector(Xvector):
    def __init__(self, input_size, lin_neurons=512, activation=nn.LeakyReLU, device = 'cpu',tdnn_channels = [512, 512, 512, 512, 1500], use_stat_pool=False):
        super().__init__(in_channels=input_size, lin_neurons=lin_neurons, activation=activation,tdnn_channels=tdnn_channels)
        self.device = torch.device(device)
        self.to(self.device)

        self.out_channels = tdnn_channels[-1]
        self.input_size = input_size
        
        if use_stat_pool:
            self.return_mean = True
            self.return_std = True
            self.blocks = nn.ModuleList([
                *self.blocks[:-2],
                CustomStatisticsPooling(),
                nn.Linear(
                    in_features= self.out_channels * (2 if self.return_mean and self.return_std else 1),  
                    out_features=lin_neurons,
                    bias=True,
                )
            ])
        else:
            self.blocks = nn.ModuleList([
                *self.blocks[:-2],
                CustomAttentionPooling(self.out_channels),
                nn.Linear(
                    in_features=self.out_channels,  # Adapt to the output size
                    out_features=lin_neurons,
                    bias=True,
                )
            ])
        
        
        self.blocks.to(self.device)
        
        self.attention_layer = self.blocks[-2]
        
    def create_mask(self, lens, max_len=None):
        if max_len is None:
            max_len=lens.max()
            
        return torch.arange(max_len, device=lens.device)[None, :] < lens[:, None]
    
    def forward(self, x, lens=None):
        if isinstance(x, list):
            #for element in x:
            x = [element.to(self.device) for element in x]
        else:
            x = x.to(self.device)
            
        mask = None
        if lens is not None:
            mask = self.create_mask(lens, x.size(1))
        
        for block in self.blocks[:-2]:
            x = block(x)
            
        x = self.attention_layer(x, mask=mask)
        x = self.blocks[-1](x)
        return x #super().forward(x, lens)
      



In [7]:
class CustomDataset(Dataset):
    def __init__(self, files, is_tv, labels: List[int], is_test= False, norm_feat = True, transform=None, target_transform=None):
        super().__init__()
        self.files = files
        
        self.labels = labels
        self.is_test = is_test
        self.is_tv = is_tv
        self.norm_feat = norm_feat
        
     
    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
                    
        #data = pd.read_pickle(os.path.join(self.dir, self.files[idx])).to_numpy()  
        
        if self.is_tv: data = pd.read_pickle(self.files[idx]).to_numpy() 
        else:
            #wav_file = pd.read_csv(self.files[idx])
            samplerate, file = wavfile.read(self.files[idx])
            feat = librosa.feature.melspectrogram(
                y=file.astype(float), 
                sr=samplerate, 
                #n_fft=400,
                #hop_length=160,
                #win_length= 400,
                #window="hamming",
                #center=False, 
                n_mels=80
                )
            #feat = librosa.feature.mfcc(y=file.astype(float), sr=samplerate)
            log_mel = librosa.power_to_db(feat)
            data = log_mel.transpose(1,0)
            #data = feat.transpose(1,0)
            
        if self.norm_feat:
            data = normalize(data, axis=0)
        
        return {'features' : data, 'label' : self.labels[idx]} 
    
class WavTVDataset(Dataset):
    def __init__(self, wav_dataset, tv_dataset):
        super().__init__()
        assert len(wav_dataset) == len(tv_dataset)
        self.wav_dataset = wav_dataset
        self.tv_dataset = tv_dataset
        
       
        
    def __len__(self):
        return len(self.wav_dataset)
    
    def __getitem__(self, idx):
        
        wav_item = self.wav_dataset[idx]
        tv_item = self.tv_dataset[idx]
        
        return {
            'wav_features': wav_item['features'],
            'tv_features': tv_item['features'],
            'label': wav_item['label']
        }



In [8]:
def val_split_speaker_ids(speaker_ids,val_ratio : float=0.1):
    
    unique_speaker_ids = list(set(speaker_ids))
    
    unique_speaker_ids.sort()
    random.shuffle(unique_speaker_ids)
    
    train_speaker_ids, val_speaker_ids = train_test_split(
        unique_speaker_ids, test_size=val_ratio, random_state=42
        )
    
    return train_speaker_ids, val_speaker_ids

def split_dataset(samples, labels, train_speaker_ids, speaker_ids):
        
    t_paths, t_labels, v_paths, v_labels = [], [], [], []
    
    for path, label, speaker_id in zip(samples, labels, speaker_ids):
        if speaker_id in train_speaker_ids:
            t_paths.append(path)
            t_labels.append(label)
        else:
            v_paths.append(path)
            v_labels.append(label)
   
    
    return t_paths, t_labels, v_paths, v_labels

In [9]:
def get_filename(file_path):
    return os.path.splitext(os.path.basename(file_path))[0]


def organize_data_in_folders(ds_type, fold):
    target_dir = f"/home/stinasb/SSL4PR/TV_train_test/Fold{fold}/{ds_type}/"

    original_dir = f"/home/stinasb/SSL4PR/pcgita_splits/TRAIN_TEST_{fold}/{ds_type}.csv"

    file = pd.read_csv(original_dir)
    
    audio_path = list(file['audio_path'])

    origin_id = list(file['original_id'])
    speaker_ids = list(file['speaker_id'])
    
    dirname = '/home/stinasb/SSL4PR/TVs_copy_test/'

    labels = []
    filelist = []
    origin_ids = []
    speaker_id = []
    
    for root, dirs, files in os.walk(dirname):
        if os.path.basename(root) in {'hc', 'pd', 'HC', 'PD'}:
            group = os.path.basename(root)
            for file in files:
                filename, extension = os.path.splitext(file)
                if filename in origin_id:
                    
                    filelist.append(os.path.join(root, file))
                    
                    
    
    wav_files_sorted = sorted(audio_path, key=get_filename)
    pkl_files_sorted = sorted(filelist, key=get_filename)
    
    for element in pkl_files_sorted:
        filename = os.path.basename(element)
        spk_id, extension = os.path.splitext(filename)
        speaker_id.append(spk_id.split("_")[0])
        
        group = os.path.basename(os.path.dirname(element))
        if group == 'hc' or group == 'HC':
            labels.append(0)
        elif group == 'pd' or group == 'PD':
            labels.append(1)
                        
        
    file_dict = {'sample_tv': pkl_files_sorted, 'sample_wav': wav_files_sorted , 'labels': labels, 'speaker_id': speaker_id}
    
    
    df = pd.DataFrame(file_dict)
    df.to_csv(target_dir + f"{ds_type}.csv", index=False)


In [10]:
class CombinedModel(nn.Module):
    def __init__(self, input_size, lin_neurons, xvec, lin_blocks = 1, classifier_out_neurons = 2, device="cpu"):
        super(CombinedModel, self).__init__()
        
        # Initialize the CustomXvector model
        self.device = device
        self.xvector = xvec.to(device)
        # Initialize the Classifier model
       
        self.classifier = None
        self.classifier_out_neurons = classifier_out_neurons
        self.device = device
        self.lin_blocks = lin_blocks
        
    def forward(self, x, lens=None):
        # Forward pass through the x-vector model
        xvector_output = self.xvector(x, lens)
        
        if self.classifier is None:
            self.classifier = Classifier(
                input_shape=xvector_output.shape,
                out_neurons=self.classifier_out_neurons,
                lin_blocks=self.lin_blocks,
            ).to(self.device)
            
        # Forward pass through the classifier
        classifier_output = self.classifier(xvector_output)
        
        return classifier_output

class WavTVCombinedModel(nn.Module):
    def __init__(self, wav_xvector, tv_xvector, mlp_hidden=256, num_classes=2, device='cpu', lin_blocks=2):
        super(WavTVCombinedModel, self).__init__()
        
        self.wav_xvector = wav_xvector.to(device)
        self.tv_xvector = tv_xvector.to(device)
        #self.mlp_hidden = mlp_hidden
        
        #self.input_dim = self.wav_xvector.blocks[-1].out_features + self.tv_xvector.blocks[-1].out_features 
        
        self.classifier = None
        self.classifier_out_neurons = num_classes
        self.device = device
        self.lin_neurons = mlp_hidden
        self.lin_blocks = lin_blocks
        
        
        
    def forward(self, wav_input, tv_input, wav_lens=None, tv_lens=None):
        
        xvec_wav = self.wav_xvector(wav_input.float(), wav_lens)
        xvec_tv = self.tv_xvector(tv_input.float(), tv_lens)
        
    
        xvec_combined = torch.cat((xvec_wav, xvec_tv), dim=1)
        
        if self.classifier is None:
            self.classifier = Classifier(
                input_shape=xvec_combined.shape,
                out_neurons=self.classifier_out_neurons,
                lin_blocks=self.lin_blocks,
                lin_neurons=self.lin_neurons,
            ).to(self.device)
        
        logits = self.classifier(xvec_combined)
        
        return logits


In [11]:
def custom_collate_fn(batch):
    # Convert each element in the batch to a tensor
    sequences_tv = [torch.tensor(item['tv_features']) for item in batch]
    lengths_tv = torch.tensor([len(seq) for seq in sequences_tv])  # Sequence lengths
    
    # Pad sequences to the same length
    padded_sequences_tv = pad_sequence(sequences_tv, batch_first=True)
    
    sequences_wav = [torch.tensor(item['wav_features']) for item in batch]
    lengths_wav = torch.tensor([len(seq) for seq in sequences_wav])  # Sequence lengths
    
    # Pad sequences to the same length
    padded_sequences_wav= pad_sequence(sequences_wav, batch_first=True)
    
    labels = torch.tensor([item['label'] for item in batch])
    
    return {'sequences_wav': padded_sequences_wav, 'sequences_tv': padded_sequences_tv,'lengths_wav': lengths_wav, 'lengths_tv':lengths_tv, 'labels': labels}


In [12]:
def data_handling(fold, val_ratio=0.1, batch_size=16, norm_feat=True):
    
    #--Organize data in folders (similar to pcgita-split)--
    organize_data_in_folders('train', fold)
    organize_data_in_folders('test', fold)

    #--Obtain data from csv-files--
    csv_path_train = f"/home/stinasb/SSL4PR/TV_train_test/Fold{fold}/train/train.csv"
    df_train = pd.read_csv(csv_path_train)

    csv_path_test = f"/home/stinasb/SSL4PR/TV_train_test/Fold{fold}/test/test.csv"
    df_test = pd.read_csv(csv_path_test)
    
    #--Organize data in samples, speaker_id and labels--
    pickle_train = df_train['sample_tv']
    pickle_test = df_test['sample_tv']
    speaker_ids = df_train['speaker_id']

    labels_train = df_train['labels']
    labels_test = df_test['labels']
        
    wav_train = df_train['sample_wav']
    wav_test = df_test['sample_wav']
             
    
    #--Split training set into a train and validation set--
    train_speaker_ids, val_speaker_ids = val_split_speaker_ids(speaker_ids, val_ratio)
    
    train_data, train_labels, val_data, val_labels = split_dataset(pickle_train, labels_train, train_speaker_ids, speaker_ids)
    train_data_wav, train_labels_wav, val_data_wav, val_labels_wav = split_dataset(wav_train, labels_train, train_speaker_ids, speaker_ids)
    
    
    #--Organize in CustomDataset objects--
    train_ds = CustomDataset(train_data, is_tv=True, labels= train_labels, is_test=False, norm_feat=norm_feat)
    val_ds = CustomDataset(val_data, is_tv=True, labels= val_labels, is_test=False, norm_feat=norm_feat)
    test_ds = CustomDataset(pickle_test, is_tv=True, labels=labels_test, is_test=True, norm_feat=norm_feat)
    
    train_ds_wav = CustomDataset(train_data_wav, is_tv=False, labels= train_labels_wav, is_test=False, norm_feat=norm_feat)
    val_ds_wav = CustomDataset(val_data_wav, is_tv=False, labels= val_labels_wav, is_test=False, norm_feat=norm_feat)
    test_ds_wav = CustomDataset(wav_test, is_tv=False, labels=labels_test, is_test=True, norm_feat=norm_feat)
    
   
    
    
    ds_train = WavTVDataset(train_ds_wav, train_ds)
    ds_val = WavTVDataset(val_ds_wav, val_ds)
    ds_test = WavTVDataset(test_ds_wav, test_ds)
    
    #--Load Datasets into Dataloaders--  
    train_dl = DataLoader(ds_train, batch_size=batch_size, collate_fn=custom_collate_fn, shuffle=True , drop_last=True)#, shuffle=True, drop_last=False)
    val_dl = DataLoader(ds_val, batch_size=batch_size, collate_fn=custom_collate_fn, shuffle=False)
    test_dl = DataLoader(ds_test, batch_size=batch_size, collate_fn=custom_collate_fn, shuffle=False)
    
  
    
    return train_dl, val_dl, test_dl
    

In [13]:
def train_one_epoch(model, train_dataloader, optimizer, scheduler, device, loss_fn, multiple_models=True, gradient_accumulation_steps=1, global_step=0, is_binary_classification=False):
    # Define a loss function and optimizer
    
    model.train()
    training_loss = 0.0
    
    p_bar = tqdm(train_dataloader, total=len(train_dataloader), ncols=100)
    
    for batch in p_bar:  # Assuming your dataloader provides (inputs, labels)
        
        if multiple_models:
            inputs_tv = batch['sequences_tv']
            inputs_wav = batch['sequences_wav']
            labels = batch['labels']
            inputs_tv, inputs_wav, labels = inputs_tv.to(device), inputs_wav.to(device), labels.to(device)
            lens_tv = batch['lengths_tv'].to(device)
            lens_wav = batch['lengths_wav'].to(device)
            
          
        # Forward pass
            outputs = model(inputs_wav, inputs_tv, lens_wav, lens_tv)
            
        else: 
            inputs_tv = batch['sequences_tv']
            labels = batch['labels']
            lens_tv = batch['lengths_tv']
            inputs_tv, labels, lens_tv = inputs_tv.to(device), labels.to(device), lens_tv.to(device)
            outputs = model(inputs_tv, lens_tv)
            
        
        
        if torch.isnan(outputs).any():
            print("Skipping batch because of nan")
            # clear memory
            del batch
            del labels
            del outputs
            torch.cuda.empty_cache()
            continue
        
        n_classes = 2
        
        if is_binary_classification: loss = loss_fn(outputs.squeeze(-1), labels) #loss_fn(outputs.squeeze(-1), labels.float())
        else: loss = loss_fn(outputs.view(-1,n_classes), labels.view(-1))

        if gradient_accumulation_steps > 1:
            loss = loss / gradient_accumulation_steps
            
        loss.backward()
        
        if (p_bar.n + 1) % gradient_accumulation_steps == 0 or p_bar.n == len(train_dataloader) - 1:
            optimizer.step()
            
            scheduler.step()
            
            optimizer.zero_grad()
       
        training_loss += loss.item()
        
        p_bar.set_postfix({"loss": loss.item(), "lr":scheduler.get_last_lr()[-1]})
        
        global_step += 1
        
        torch.cuda.empty_cache()

    return training_loss / len(train_dataloader), global_step

In [14]:
def evaluate_model(model, eval_dataloader, device, loss_fn, multiple_models=True, is_binary_classification=False):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []
    
    p_bar = tqdm(eval_dataloader, total=len(eval_dataloader), ncols=100)
    eval_loss = 0.0
    reference, predictions = [], []

    with torch.no_grad():  # Disable gradient computation
        for batch in p_bar:  # Assuming (inputs, labels, lengths) from the dataloader
            if multiple_models:
                inputs_tv = batch['sequences_tv']
                inputs_wav = batch['sequences_wav']
                labels = batch['labels']
                lens_tv = batch['lengths_tv']
                lens_wav = batch['lengths_wav']
                
                inputs_tv, lens_tv, labels = inputs_tv.to(device), lens_tv.to(device), labels.to(device)
                inputs_wav, lens_wav = inputs_wav.to(device), lens_wav.to(device)

                # Forward pass
                outputs = model(inputs_wav, inputs_tv, lens_wav, lens_tv)
            else: 
                inputs_tv = batch['sequences_tv']
                labels = batch['labels']
                lens_tv = batch['lengths_tv']
                
                inputs_tv, lens_tv, labels = inputs_tv.to(device), lens_tv.to(device), labels.to(device)
                
                outputs = model(inputs_tv, lens_tv)
            
            n_classes = outputs.shape[-1]
            
            if is_binary_classification: loss = loss_fn(outputs.squeeze(-1), labels)
            else: loss = loss_fn(outputs.view(-1, n_classes), labels.view(-1))
            
            eval_loss += loss.item()
            reference.extend(labels.cpu().numpy())
            if is_binary_classification: predictions.extend((outputs > 0.5).cpu().numpy().astype(int))
            else: predictions.extend(torch.argmax(outputs, dim=-1).cpu().numpy().astype(int))
            
            p_bar.set_postfix({"loss": loss.item()})
    return eval_loss / len(eval_dataloader), reference, predictions
            



In [15]:
def compute_metrics(reference, predictions, verbose=False, is_binary_classification=False):
    accuracy = accuracy_score(reference, predictions)
    
    report = classification_report(reference, predictions, output_dict=True)
    
    return accuracy, report

In [13]:
torch.cuda.empty_cache()

In [20]:
folds = 10
val_ratio = 0.1
batch_size = 8
norm_feat = True
use_stat_pool = False
acc_per_test_fold = {}

checkpoint_path = f"/home/stinasb/SSL4PR/checkpoints/tv_testing"

for test_fold in range(1, folds+1):
    
    input_size = 9
    device = "cuda:3" if torch.cuda.is_available() else "cpu"
   
    train_dl, val_dl, test_dl = data_handling(test_fold, val_ratio=val_ratio, batch_size=batch_size, norm_feat=norm_feat)

    #print(torch.cuda.memory_allocated())
    #print(torch.cuda.memory_reserved())
    # Combine them into a single model
    wav_xvec = CustomXvector(input_size=80, device=device, use_stat_pool=use_stat_pool)
    tv_xvec = CustomXvector(input_size=9, device=device, use_stat_pool=use_stat_pool)
   
    
    
    #model = CombinedModel(input_size=input_size, lin_neurons=512, xvec=tv_xvec,lin_blocks=1, classifier_out_neurons=2, device=device)
    model = WavTVCombinedModel(wav_xvec, tv_xvec, mlp_hidden=1024, num_classes=2, device=device, lin_blocks=1)

    
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=0.01)
    num_epochs = 14
    gradient_accumulation_step = 8

    total_steps = int(len(train_dl)*num_epochs) // gradient_accumulation_step
    warmup_ratio = 0.1

    scheduler = get_linear_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=int(total_steps*warmup_ratio),
        num_training_steps=total_steps,
        last_epoch=-1,
    )

    is_binary_classification = False
    multiple_models = True

    if is_binary_classification: loss_fn = nn.BCELoss()
    else: loss_fn = nn.CrossEntropyLoss()

    global_step = 0
    best_val_accuracy = 0
    

    for epoch in range(num_epochs):
        training_loss, global_step = train_one_epoch(
            model=model,
            train_dataloader=train_dl,
            optimizer=optimizer,
            scheduler=scheduler,
            device=device,
            loss_fn=loss_fn,
            multiple_models=multiple_models,
            gradient_accumulation_steps=gradient_accumulation_step,
            global_step=global_step,
            is_binary_classification=is_binary_classification,
        )
        
        print(global_step)
        
        print(f"Average training loss: {training_loss}")
        
        val_loss, val_reference, val_predictions = evaluate_model(
            model=model,
            eval_dataloader=val_dl,
            device=device,
            loss_fn=loss_fn,
            multiple_models=multiple_models,
            is_binary_classification=is_binary_classification,
        )
        print(f"Average validation loss: {val_loss}")

        val_acc, val_report = compute_metrics(val_reference, val_predictions)
        print(f"Validation accuracy: {val_acc}")
        
        if val_acc > best_val_accuracy:
            print(f"Found a better model with accuracy: {val_acc:.3f} - previous best: {best_val_accuracy:.3f}")
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), checkpoint_path + f"/fold_{test_fold}.pt")
        
        
    model.load_state_dict(torch.load(checkpoint_path + f"/fold_{test_fold}.pt", weights_only=True))
 
    test_loss, test_reference, test_predictions = evaluate_model(
        model, test_dl, device=device, loss_fn=loss_fn, multiple_models=multiple_models)
    
    test_acc, test_report = compute_metrics(test_reference, test_predictions)
    #print(f"Testing accuracy: {test_acc}")
    print(f"Testing accuracy (testfold {test_fold}): {test_acc}")
    acc_per_test_fold[test_fold] = test_acc
    
    
    
    
    torch.cuda.empty_cache()

    
results_df = pd.DataFrame(acc_per_test_fold, index=[0])
    
results_df.to_csv("/home/stinasb/SSL4PR/TVs_copy" + "/test_results.csv", index=False)

100%|███████████████████████████████████| 182/182 [00:35<00:00,  5.19it/s, loss=0.0796, lr=0.000256]


182
Average training loss: 0.0870332637658486


100%|████████████████████████████████████████████████████| 21/21 [00:04<00:00,  4.95it/s, loss=0.63]


Average validation loss: 0.7751839657624563
Validation accuracy: 0.49382716049382713
Found a better model with accuracy: 0.494 - previous best: 0.000


100%|██████████████████████████████████████████| 182/182 [00:38<00:00,  4.69it/s, loss=0.0747, lr=0]


...

364
Average training loss: 0.08301831122774345


100%|███████████████████████████████████████████████████| 21/21 [00:03<00:00,  5.30it/s, loss=0.379]


Average validation loss: 0.6043155633267903
Validation accuracy: 0.7222222222222222
Found a better model with accuracy: 0.722 - previous best: 0.623


100%|███████████████████████████████████████████████████| 23/23 [00:04<00:00,  4.92it/s, loss=0.589]

Testing accuracy (testfold 2): 0.6222222222222222


In [3]:
df = pd.read_csv("/home/stinasb/SSL4PR/TVs_copy/test_results.csv")
df

,1,2,3,4,5,6,7,8,9,10
0,0.633333,0.75,0.805556,0.7,0.583333,0.811111,0.588889,0.677778,0.692737,0.672222


In [4]:
print(f"mean: {np.mean(df.values)}")
print(f"std: {np.std(df.values)}")

mean: 0.691495965238982
std: 0.07548766775842185


In [1]:
path = "/home/stinasb/SSL4PR/TV_train_test/Fold1/train/train.csv"

df = pd.read_csv(path)

df[0]

NameError: name 'pd' is not defined